## Project 1 - SQL Exploration

### Menampilkan data yang ada di dalam tabel covid_deaths dan covid_vaccination

In [0]:
-- Menampilkan data covid_deaths
SELECT * FROM project_covid.covid_deaths ORDER BY 3, 4 

In [0]:
SELECT * FROM project_covid.covid_vaccinations ORDER BY 3, 4

### Data yang akan digunakan 

In [0]:
-- Beberapa kolom yang akan digunakan dalam eksplorasi diantaranya : location (negara), date, total_cases, new_cases per harinya. total_death, serta population dalam negara tersebut
SELECT location, date, total_cases, new_cases, total_deaths, population
FROM project_covid.covid_deaths 
WHERE continent IS NOT NULL 
ORDER BY 1,2 

### Case No. 1 - Total Cases vs Total Deaths
Memberikan persentase peluang meninggal jika terinfeksi covid dalam sebuah negara 

In [0]:
SELECT location, date, total_cases, new_cases, (total_deaths/total_cases)*100 as DeathPercentage
FROM project_covid.covid_deaths 
WHERE continent IS NOT NULL
ORDER BY 1,2 

### Case No.2 - Total Case VS Population 
1. Melihat persentase populasi orang yang terinfeksi covid dalam sebuah negara
2. Menampilkan negara dengan angka infeksi tertinggi apabila dibandingkan dengan populasinya
3. Negara dengan angka kematian tertinggi
4. Benua dengan angka kematian tertinggi

In [0]:
-- 1.
SELECT location, date, population, total_cases, (total_cases/population)*100 AS PercentPupulationInfected
FROM project_covid.covid_deaths
WHERE continent IS NOT NULL
ORDER BY 1,2

In [0]:
-- 2.
SELECT location, population, MAX(total_cases) AS HighestInfectionCount, MAX(total_cases)*100/population AS PercentPopulationInfected
FROM project_covid.covid_deaths
GROUP BY location, population
ORDER BY PercentPopulationInfected DESC

In [0]:
-- 3.
SELECT location, MAX(cast(total_deaths as int)) AS TotalDeathCount
From project_covid.covid_deaths
WHERE continent IS NOT NULL
GROUP BY location
ORDER BY TotalDeathCount DESC

In [0]:
-- 4.
SELECT continent, MAX(cast(total_deaths as int)) AS TotalDeathCount
From project_covid.covid_deaths
WHERE continent IS NOT NULL
GROUP BY continent
ORDER BY TotalDeathCount DESC

### Case No.3 - Persentase Kematian secara Global

In [0]:
SELECT SUM(new_cases) as total_cases, 
SUM(cast(new_deaths as int)) as total_deaths, 
SUM(cast(new_deaths as int))/SUM(new_cases)*100 as DeathPercentage
FROM project_covid.covid_deaths
WHERE continent IS NOT NULL

### Case No.4 - Total Population VS Vaccinations (JOIN CASE)
Persentase populasi yang telah melakukan vaksinasi covid dalam sebuah negara

PS: Di dalam Spark SQL (SQL yang digunakan di dalam Databrick tidak mengenal perintah CONVERT sehingga pada syntax dibawah tetap menggunakan CAST)

In [0]:
SELECT death.continent, death.location, death.date, death.population, 
vac.new_vaccinations, SUM(CAST(vac.new_vaccinations as INT)) OVER (PARTITION BY death.location ORDER BY death.location, death.date) AS RollingPeopleVaccinated
FROM project_covid.covid_deaths death
JOIN project_covid.covid_vaccinations vac
ON death.location = vac.location AND death.date = vac.date
WHERE death.continent IS NOT NULL
ORDER BY 2,3

#### CTE 

In [0]:
-- Using CTE to perform Calculation on Partition By in previous query 

With PopvsVac (Continent, Location, Date, Population, New_Vaccinations, RollingPeopleVaccinated)
AS 
(
SELECT death.continent, death.location, death.date, death.population, vac.new_vaccinations, SUM(CAST(vac.new_vaccinations as INT)) OVER (PARTITION BY death.location ORDER BY death.location, death.date) AS RollingPeopleVaccinated
FROM project_covid.covid_deaths death
JOIN project_covid.covid_vaccinations vac
ON death.location = vac.location AND death.date = vac.date
WHERE death.continent IS NOT NULL
)

SELECT *, (RollingPeopleVaccinated/Population)*100
FROM PopvsVac


#### View

In [0]:
-- Creating View to store data for later visualizations
CREATE VIEW PercentPopulationVaccinated AS
SELECT death.continent, death.location, death.date, death.population, vac.new_vaccinations, SUM(CAST(vac.new_vaccinations as INT)) OVER (PARTITION BY death.location ORDER BY death.location, death.date) AS RollingPeopleVaccinated
FROM project_covid.covid_deaths death
JOIN project_covid.covid_vaccinations vac
ON death.location = vac.location AND death.date = vac.date
WHERE death.continent IS NOT NULL

In [0]:
SELECT *, (RollingPeopleVaccinated/Population)*100 
FROM PercentPopulationVaccinated